In [58]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [59]:
import numpy as np
import os
import time

In [60]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [61]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))# Take a look at the first 250 characters in text
print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [62]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [63]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [27]:
text_as_int.shape

(1115394,)

In [64]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [65]:
#splitting in sequence


seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)



In [32]:
# text_as_int

array([18, 47, 56, ..., 45,  8,  0])

<h3> Dividing stuff into 101 data tensors </h3>


In [66]:
# char_dataset
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [34]:
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [67]:
for item in sequences.take(2):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


<h3>Creating test and train data each of 100 character large </h3>


In [68]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [41]:
for input_example, target_example in  dataset.take(2):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
  print()

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '

Input data:  'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
Target data: 're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'



In [90]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

datas1t = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

datas1t


# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [91]:
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#   model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                               batch_input_shape=[batch_size, None]),
#     tf.keras.layers.GRU(rnn_units,
#                         return_sequences=True,
#                         stateful=True,
#                         recurrent_initializer='glorot_uniform'),
#     tf.keras.layers.Dense(vocab_size)
#   ])
#   return model


# Usage of lSTm instead of GRated 
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,embedding_dim, rnn_units, BATCH_SIZE)
model.summary()



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_4 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_6 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [71]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [53]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [83]:
for input_example_batch, target_example_batch in datas1t.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape


print(len(example_batch_predictions))
# print(example_batch_predictions.shape)
pred = example_batch_predictions[0]
# print(len(pred))
# print(pred)
time_pred = pred[0]
# print(len(time_pred))
# print(time_pred)

# For each batch of 64 it reads all 100 characters and after reading each character its makes a prediction each of 65 characters length distribution which tells probability of each character occurence



(64, 100, 65) # (batch_size, sequence_length, vocab_size)
64


In [84]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174895


In [94]:
model.compile(optimizer='adam', loss=loss)

In [92]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [95]:
EPOCHS=190
# history = model.fit(datas1t, epochs=EPOCHS )
history = model.fit(data, epochs=150, callbacks=[checkpoint_callback])

Epoch 1/150
172/172 [==============================] - 29s 167ms/step - loss: 2.5324
Epoch 2/150
172/172 [==============================] - 28s 166ms/step - loss: 1.8478
Epoch 3/150
172/172 [==============================] - 29s 166ms/step - loss: 1.6092
Epoch 4/150
172/172 [==============================] - 29s 166ms/step - loss: 1.4813
Epoch 5/150
172/172 [==============================] - 29s 166ms/step - loss: 1.4044
Epoch 6/150
172/172 [==============================] - 29s 166ms/step - loss: 1.3487
Epoch 7/150
172/172 [==============================] - 29s 166ms/step - loss: 1.3043
Epoch 8/150
172/172 [==============================] - 29s 166ms/step - loss: 1.2643
Epoch 9/150
172/172 [==============================] - 29s 166ms/step - loss: 1.2275
Epoch 10/150
172/172 [==============================] - 29s 166ms/step - loss: 1.1915
Epoch 11/150
172/172 [==============================] - 29s 166ms/step - loss: 1.1535
Epoch 12/150
172/172 [==============================] - 29s 166

In [96]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [100]:
model1 = build_model(VOCAB_SIZE,embedding_dim
                     , rnn_units, batch_size=1)

model1.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model1.build(tf.TensorShape([1, None]))




In [101]:
# print(generate_text(model, start_string=u"ROMEO: "))
inp = input("Type a starting string: ")
print(generate_text(model1, inp))



Type a starting string: ROMEO
ROMEONES:
Your grace I see, signior Baptista, I'll adventure
Turn to denyown these sad storemost:
Thy other Edward dead, to eace, Henry, and England,
Thou may'st himself with hate, that humble prophecond this my name!

HASTINGS:

Messenger:
A man that loves to her heart will live
Till then our person that might thrust appear,
Put you out of fourses, nothing nurse, be not
be, thence for Reventer's right:
But thou didst kill our very meaning,
One word, you and your confirmation to the Capitol!

All:
Call Cophelland's king, who think I do not know--
Whethails old mannersome us to weep.

CLARENCE:
O, no doubt, rich in Gloucester all Northumberland,
I will not sun
See, Biondello, he did but right day that,
Which, too much I ripe? and Marcius does,
I see that rather feel'd and shake no monster to the Tower,
Turn theest thou sweet nate
My father's right days, benevore high tractors for a guest
Is adgly first in grace and tyranny
The bloody office lad worlike pity

In [9]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np


Using TensorFlow backend.


In [3]:
def plot_graph(model_y,metric):
  plt.plot(model_y.history[metric])
  plt.plot(model_y.history['val_'+metric],'')
  plt.xlabel("Epochs")
  plt.ylabel(metric)

  plt.legend([metric, 'val_'+metric])
  plt.show()


  pass


In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,as_supervised=True)
train_dataset , test_dataset = dataset['train'],dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePT6I6B/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePT6I6B/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePT6I6B/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [5]:
encoder = info.features['text'].encoder

In [6]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [7]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [8]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [11]:
assert original_string == sample_string
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))



4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [13]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)


<h2>  An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors. </h2>


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [19]:
encoder.vocab_size

8185

In [18]:
history = model.fit(train_dataset, epochs=20,
                    validation_data=test_dataset )
                    # ,validation_steps=30)

Epoch 1/20


ValueError: ignored